In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-bayanzrh/'   
    
    print("Loading page...")
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    all_listing_urls = set()
    current_page = 1
    max_pages = 5  # Reduced to 5 for faster testing
    no_new_count = 0
    
    while current_page <= max_pages:
        print(f"\nPage {current_page}/{max_pages}")
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        # ✅ FIX: Actually check and add new listings!
        new_listings = page_listings - all_listing_urls
        print(f"  Found: {len(page_listings)} | New: {len(new_listings)}")
        
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                print("No more new listings, stopping")
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
            print(f"  Total collected: {len(all_listing_urls)}")
        
        # Navigate to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    print(f"\n{'='*50}")
    print(f"Total listings collected: {len(listing_urls)}")
    print(f"{'='*50}")
    
    if len(listing_urls) == 0:
        print("No listings found")
        driver.quit()
        exit()
    
    response = input(f"\nScrape details for {len(listing_urls)} listings? (y/n): ")
    if response.lower() != 'y':
        driver.quit()
        exit()
    
    print("\nScraping details...")
    start_time = time.time()
    successful = 0
    
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            if i % 10 == 0:
                elapsed = (time.time() - start_time) / 60
                rate = i / (time.time() - start_time) * 60
                remaining = (len(listing_urls) - i) / rate if rate > 0 else 0
                print(f"[{i}/{len(listing_urls)}] {elapsed:.1f}m | {rate:.1f}/min | ~{remaining:.1f}m left")
            
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []
            
            key_finder('Шал:', 0, span_list, key_list)
            key_finder('Тагт:', 1, span_list, key_list)
            key_finder('Гараж:', 2, span_list, key_list)
            key_finder('Цонх:', 3, span_list, key_list)
            key_finder('Хаалга:', 4, span_list, key_list)
            key_finder('Цонхнытоо:', 5, span_list, key_list)
            key_finder('Барилгынявц', 6, span_list, key_list)
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list)
            key_finder('Барилгындавхар:', 8, span_list, key_list)
            key_finder('Талбай:', 9, span_list, key_list)
            key_finder('Хэдэндавхарт:', 10, span_list, key_list)
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list)
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list)
            
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"
            
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            key_list[16] = date.today().strftime("%Y/%m/%d")
            
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"
            
            key_list[18] = listing_url
            
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"
            
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"
            
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"
            
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"
            
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            print(f"  Error {i}: {str(e)[:50]}")
            continue
    
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(f"\n{'='*50}")
    print(f"Done! Scraped {successful}/{len(listing_urls)} listings")
    print(f"Saved to: {filename}")
    print(f"{'='*50}\n")
    print(df.head())
    
except Exception as e:
    print(f"\nError: {e}")
    
finally:
    driver.quit()

Loading page...

Page 1/5
  Found: 60 | New: 60
  Total collected: 60

Page 2/5
  Found: 60 | New: 60
  Total collected: 120

Page 3/5
  Found: 60 | New: 60
  Total collected: 180

Page 4/5
  Found: 60 | New: 60
  Total collected: 240

Page 5/5
  Found: 60 | New: 60
  Total collected: 300

Total listings collected: 300



Scrape details for 300 listings? (y/n):  y



Scraping details...
[10/300] 0.8m | 12.1/min | ~23.9m left
[20/300] 1.6m | 12.2/min | ~22.9m left
[30/300] 2.4m | 12.7/min | ~21.3m left
[40/300] 3.3m | 12.2/min | ~21.3m left
[50/300] 4.0m | 12.4/min | ~20.1m left
[60/300] 4.8m | 12.5/min | ~19.2m left
[70/300] 5.7m | 12.4/min | ~18.6m left
[80/300] 6.5m | 12.4/min | ~17.7m left
[90/300] 7.2m | 12.5/min | ~16.8m left
[100/300] 7.9m | 12.6/min | ~15.8m left
[110/300] 8.7m | 12.7/min | ~15.0m left
[120/300] 9.4m | 12.8/min | ~14.1m left
[130/300] 10.1m | 12.8/min | ~13.2m left
[140/300] 10.9m | 12.9/min | ~12.4m left
[150/300] 11.7m | 12.9/min | ~11.7m left
[160/300] 12.4m | 12.9/min | ~10.9m left
[170/300] 13.1m | 12.9/min | ~10.0m left
[180/300] 13.9m | 13.0/min | ~9.2m left
[190/300] 14.6m | 13.0/min | ~8.5m left
[200/300] 15.3m | 13.0/min | ~7.7m left
[210/300] 16.2m | 13.0/min | ~6.9m left
[220/300] 17.0m | 12.9/min | ~6.2m left
[230/300] 17.8m | 12.9/min | ~5.4m left
[240/300] 18.6m | 12.9/min | ~4.6m left
[250/300] 19.3m | 13.0/

In [3]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import csv
import time
import pandas as pd
from datetime import date
import re

header = ['Шал:', 'Тагт:', 'Гараж:', 'Цонх:', 'Хаалга:', 'Цонхнытоо:', 'Барилгынявц', 
          'Ашиглалтандорсонон:', 'Барилгындавхар:', 'Талбай:', 'Хэдэндавхарт:', 
          'Цахилгааншаттайэсэх:', 'Төлбөрийннөхцөл:', 'Дүүрэг:', 'Байршил:', 'Үзсэн:', 
          'Scraped_date:', 'Posted_date:', 'Link:', 'Үнэ:', 'Өрөөний Тоо:', 
          'Зарын гарчиг:', 'Зарын Тайлбар:']

csv_list = []

def key_finder(key, index, span_list, key_list):
    for item in span_list:
        if str(key) in item:
            key_list[index] = item.split(':')[1]
    if type(key_list[int(index)]) != str:
        key_list[int(index)] = "Мэдээлэл байхгүй"

def extract_listing_links(soup):
    listing_links = set()
    pattern = re.compile(r'^/adv/\d+_[a-z0-9-]+/?$')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if pattern.match(href):
            if not any(exclude in href for exclude in ['?', '#', 'page=', 'sort=', 'view=']):
                listing_links.add(href)
    
    return listing_links

options = uc.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = uc.Chrome(options=options, version_main=None) 
wait = WebDriverWait(driver, 15)

try:
    base_url = 'https://www.unegui.mn/l-hdlh/l-hdlh-zarna/oron-suuts-zarna/ub-bayanzrh/'   
    
    driver.get(base_url)
    time.sleep(5)  
    
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    all_listing_urls = set()
    current_page = 1
    max_pages = 2  
    no_new_count = 0
    
    while current_page <= max_pages:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_listings = extract_listing_links(soup)
        
        new_listings = page_listings - all_listing_urls
        
        if len(new_listings) == 0:
            no_new_count += 1
            if no_new_count >= 2:
                break
        else:
            no_new_count = 0
            all_listing_urls.update(new_listings)
        
        # to next page
        if current_page < max_pages:
            next_page_url = f"{base_url}?page={current_page + 1}"
            driver.get(next_page_url)
            time.sleep(4)
            
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='/adv/']")))
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            current_page += 1
        else:
            break
    
    listing_urls = [f"https://www.unegui.mn{link}" for link in all_listing_urls]
    
    if len(listing_urls) == 0:
        driver.quit()
        exit()
    
    # scraping
    start_time = time.time()
    successful = 0
    
    for i, listing_url in enumerate(listing_urls, 1):
        try:
            driver.get(listing_url)
            time.sleep(1.5)
            
            wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            key_list = ["Мэдээлэл байхгүй"] * len(header)
            
            try:
                details = soup.find('div', class_='announcement-characteristics clearfix').find_all('li')
                span_list = [item.text.replace('\n', '').replace(' ', '') for item in details]
            except:
                span_list = []
            
            key_finder('Шал:', 0, span_list, key_list) #floor
            key_finder('Тагт:', 1, span_list, key_list) #balcony
            key_finder('Гараж:', 2, span_list, key_list) #garage
            key_finder('Цонх:', 3, span_list, key_list) #window
            key_finder('Хаалга:', 4, span_list, key_list) #door
            key_finder('Цонхнытоо:', 5, span_list, key_list) #number of windows
            key_finder('Барилгынявц', 6, span_list, key_list) #construction progress
            key_finder('Ашиглалтандорсонон:', 7, span_list, key_list) #built year
            key_finder('Барилгындавхар:', 8, span_list, key_list) #total floor
            key_finder('Талбай:', 9, span_list, key_list) #size
            key_finder('Хэдэндавхарт:', 10, span_list, key_list) #located floor
            key_finder('Цахилгааншаттайэсэх:', 11, span_list, key_list) #elevator
            key_finder('Төлбөрийннөхцөл:', 12, span_list, key_list) #payment term
            
            try:
                address = soup.find('span', itemprop="address").text.split('—')
                key_list[13] = address[0].strip() if len(address) > 0 else "Мэдээлэл байхгүй"
                key_list[14] = address[1].strip() if len(address) > 1 else "Мэдээлэл байхгүй"
            except:
                key_list[13] = key_list[14] = "Мэдээлэл байхгүй"
            
            try:
                key_list[15] = soup.find('span', class_='counter-views').text.strip()
            except:
                key_list[15] = "Мэдээлэл байхгүй"
            
            key_list[16] = date.today().strftime("%Y/%m/%d")
            
            try:
                key_list[17] = soup.find('span', class_='date-meta').text.strip()
            except:
                key_list[17] = "Мэдээлэл байхгүй"
            
            key_list[18] = listing_url
            
            try:
                key_list[19] = soup.find('meta', itemprop='price')['content']
            except:
                key_list[19] = "Мэдээлэл байхгүй"
            
            try:
                key_list[20] = soup.find('div', class_='wrap js-single-item__location').find_all('span')[-1].text.strip()
            except:
                key_list[20] = "Мэдээлэл байхгүй"
            
            try:
                key_list[21] = soup.find('h1', class_='title-announcement').text.strip()
            except:
                key_list[21] = "Мэдээлэл байхгүй"
            
            try:
                key_list[22] = soup.find('div', class_='announcement-description').text.strip()
            except:
                key_list[22] = "Мэдээлэл байхгүй"
            
            csv_list.append(dict(zip(header, key_list)))
            successful += 1
            
        except Exception as e:
            continue
    
    filename = f"Unegui_bayanzurkh_{date.today().strftime('%Y%m%d')}.csv"
    with codecs.open(filename, 'w', 'utf-8-sig') as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        writer.writerows(csv_list)
    
    df = pd.DataFrame(csv_list)
    print(df.head())
    
except Exception as e:
    pass
    
finally:
    driver.quit()

     Шал:      Тагт:   Гараж:  Цонх: Хаалга: Цонхнытоо:      Барилгынявц  \
0  Паркет   1тагттай   Байгаа  Вакум   Төмөр          3  Ашиглалтадорсон   
1  Паркет   2тагттай  Байхгүй  Вакум  Бүргэд          2  Ашиглалтадорсон   
2  Паркет  3+тагттай   Байгаа  Вакум   Төмөр          8  Ашиглалтадорсон   
3  Паркет   1тагттай  Байхгүй  Вакум     Мод          2  Ашиглалтадорсон   
4  Паркет   2тагттай   Байгаа  Вакум  Бүргэд          3  Ашиглалтадорсон   

  Ашиглалтандорсонон: Барилгындавхар:  Талбай:  ... Дүүрэг:  \
0                2019              16  74.15м²  ...      УБ   
1                2005               6   49.7м²  ...      УБ   
2                2007              13  356.0м²  ...      УБ   
3                2013               5   32.0м²  ...      УБ   
4                2022              13  76.77м²  ...      УБ   

                       Байршил:       Үзсэн: Scraped_date:  \
0  Баянзүрх, Баянзүрх, Хороо 26   Үзсэн : 48    2025/11/12   
1          Баянзүрх, Зүүн 4 зам  Үзсэн :